In [1]:
from graphdatascience import GraphDataScience
import pandas as pd
import configparser

### Neo4j Settings
The NEO4J_PROPERTIES_FILE is an ini configuration file for Neo4j properties so this notebook can connect to your Neo4j instance and load data. The ini file should be formatted as follows

```
[NEO4J]
PASSWORD=<password>
USERNAME=<username, is 'neo4j' by default>
HOST=<host uri>
```

In [2]:
config = configparser.RawConfigParser()
config.read('/Users/zachblumenfeld/devtools/aura-mind-2.ini')
HOST = config['NEO4J']['HOST']
USERNAME = config['NEO4J']['USERNAME']
PASSWORD = config['NEO4J']['PASSWORD']

gds = GraphDataScience(HOST, auth=(USERNAME, PASSWORD), aura_ds=True)

### Clean Up from Last Time

In [3]:
# remove old rels
_ = gds.run_cypher('MATCH (:RecentNews)-[r:USERS_ALSO_LIKED]->() DELETE r')

In [4]:
# remove properties
_ = gds.run_cypher('MATCH (n:News) REMOVE n.embedding')

In [5]:
# drop all the projected graphs
g_names = gds.graph.list().graphName.tolist()

for g_name in g_names:
    g = gds.graph.get(g_name)
    gds.graph.drop(g)

### Stage Graph

In [6]:
# create recent news label
gds.run_cypher('''
    MATCH(n:News)<-[:CLICKED|HISTORICALLY_CLICKED]-() WHERE n.approxTime IS NOT NULL
    SET n:RecentNews
    RETURN count(DISTINCT n)
    ''')

,count(DISTINCT n)
0,21858
